In [1]:
from datasets import load_dataset, disable_caching
from datetime import datetime as dt


import const
import json
import os
import shutil
import traceback
import pandas as pd

In [2]:
if os.path.isdir(const.TEMP_FOLDER) is False:
    os.makedirs(const.TEMP_FOLDER)

In [3]:
ds = load_dataset("stingning/ultrachat")

Loading dataset shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [4]:
train_data = ds['train']['data']

In [5]:
def convert_ultrachat_to_csv(training_data, batch_size=4000):
    """This function converts the training_data list to CSV file.
    PARAMETERS
    training_data - The list of training data to convert to CSV file.
    """
    if batch_size is None:
        new_batch_size = len(training_data)
    else:
        new_batch_size = batch_size
    
    training_data_len = len(training_data)
    start_time = dt.now()
    item_number = 0
    output_name = const.DATASETS_FOLDER + "ultrachat_training_ds.csv"
    end_time = None
    last_progress = 0
    batch_df = None
    batch_number = 0
    for item in train_data:
        
        progress = int((item_number/ training_data_len)*100)
        progress_str = ("Converting Ultrachat to CSV (" + str(progress)+ "%, Sample: " + str(item_number) + 
            "/" + str(training_data_len) +
           ", Elapsed Time: ")
        if end_time is None:
            progress_str += "00:00.00)"
        else:
            elapsed_time = end_time - start_time
            items_remaining = training_data_len - item_number
            average_processing_time = elapsed_time / item_number
            time_remaining = average_processing_time * items_remaining
            progress_str += str(elapsed_time) + ", Time Remaining: " + str(time_remaining) + ")"
        print(progress_str, end="                                                                               \r")
        question = item[0]
        temp_df = pd.DataFrame(data={"question": [question], "data": [str(item)]})
        
        if batch_df is None:
            batch_df = temp_df
        else:
            batch_df = pd.concat([batch_df, temp_df], ignore_index=True)
        item_number += 1

        if (item_number != 0):
            if (item_number % new_batch_size == 0):
                try:   
                    if batch_size is not None:
                        output_name = const.TEMP_FOLDER + "ultrachat_training_ds_batch_" + str(batch_number) + ".csv"
                    batch_df.to_csv(output_name)
                    batch_df = None
                except Exception as e:
                    batch_df = None
                    print("\nWarning: Unable to export batch " + str(batch_number) + ".")
                    print(e)
                    print("\n")
                batch_number += 1
        end_time = dt.now()
        last_progress = progress
        
    print("                                                                                                              ")
    print("Converted Ultrachat to CSV file(s).")
    

In [6]:
convert_ultrachat_to_csv(train_data, 50000)

Converting Ultrachat to CSV (57%, Sample: 849999/1468352, Elapsed Time: 0:14:16.158623, Time Remaining: 0:10:22.681471)                                                                               
need to escape, but no escapechar set


Converting Ultrachat to CSV (64%, Sample: 949999/1468352, Elapsed Time: 0:16:04.081228, Time Remaining: 0:08:46.128295)                                                                               
need to escape, but no escapechar set


Converting Ultrachat to CSV (68%, Sample: 999999/1468352, Elapsed Time: 0:16:59.671121, Time Remaining: 0:07:57.720060)                                                                               
need to escape, but no escapechar set


Converting Ultrachat to CSV (71%, Sample: 1049999/1468352, Elapsed Time: 0:17:50.569537, Time Remaining: 0:07:06.720060)                                                                               
need to escape, but no escapechar set


Converting Ultrachat to CSV (74%, Sample: 1

In [10]:
ultrachat_partitions = os.listdir(const.TEMP_FOLDER)
training_ultrachat_df = None
for partition in ultrachat_partitions:
    temp_df = pd.read_csv((const.TEMP_FOLDER + partition))
    if training_ultrachat_df is None:
        training_ultrachat_df = temp_df
    else:
        training_ultrachat_df = pd.concat([training_ultrachat_df, temp_df], ignore_index=True)

training_ultrachat_df.to_csv(const.DATASETS_FOLDER + "ultrachat_training.csv")
shutil.rmtree(const.TEMP_FOLDER)